In [1]:
import pandas as pd
import os

In [2]:
DATASET_DIR = r'D:\repo\stonks\data\dataset'

def file_path(sym, type, file_type = 'csv', dir = DATASET_DIR):
    return os.path.join(dir, f'{sym}_{type}.{file_type}')

sym = 'AAPL'
input_df = pd.read_csv(file_path(sym, 'inputs'))
output_df = pd.read_csv(file_path(sym, 'outputs'))
# input_df.columns, output_df.columns

In [3]:
from sklearn.preprocessing import maxabs_scale
import numpy as np
print(input_df.shape)
input_df.dropna(inplace=True, axis=1)
print(input_df.shape)
input = input_df.set_index('open').drop(columns=['Unnamed: 0']).values
normalized_input = maxabs_scale(input, axis=0)
# normalized_input = input / input.max(axis=0)
output_h = output_df['highest_in_5_days'].values
output_l = output_df['lowest_in_5_days'].values
def lenient_accuracy(y_true, y_pred, threshold=0.02):
    '''
    y_true: true values
    y_pred: predicted values
    threshold: threshold for leniency
    
    returns the percentage of predictions that are within the threshold 
    '''
    return np.mean((y_true - y_pred) <= threshold)

def lenient_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    return lenient_accuracy(y, y_pred)

(1277361, 94)
(1277361, 75)


In [4]:
from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(normalized_input, output_h, test_size=0.2, random_state=42)

from sklearn.linear_model import LinearRegression
# import all regression models for continuous output
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
# fitted_model = LinearRegression().fit(x_train, y_train)
# fitted_model.score(x_test, y_test)

from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    DecisionTreeRegressor(),
    ExtraTreeRegressor(),
    HistGradientBoostingRegressor(),
    # SVR(),
    # AdaBoostRegressor(), # 11470.01s = 3.2 hours
    # RandomForestRegressor(), # takes too long, 89285.37s = 24.8 hours = 1 day
    # GradientBoostingRegressor(), # takes too long, 53585.16s = 14.8 hours = 1 day
    # KNeighborsRegressor(), # takes too long,  5299/2s = 88.3 minutes = 1.5 hours
    # MLPRegressor(), # takes too long,  26224/2s = 437 minutes = 7.3 hours
    # BaggingRegressor(), # takes too long,  5684/2s = 94.7 minutes = 1.5 hours
] 
import time
import pickle
from joblib import dump, load
from pathlib import Path
results = []
# for model in models:
k_fold = KFold(n_splits=5, shuffle=False)
#     score_k = cross_val_score(LinearRegression(), x_train, y_train, cv=k_fold, n_jobs=1, scoring=lenient_scorer)
save = True
datasets = {'highs': output_h, 'lows': output_l}
for prediction_type, output in datasets.items():
    x_train, x_test, y_train, y_test = train_test_split(input, output, test_size=0.2, random_state=42)
    # y_train = y_train.reshape(-1, 1)
    # y_test = y_test.reshape(-1, 1)
    # x_train = x_train.astype(np.float16)
    # x_test = x_test.astype(np.float16)
    # y_train = y_train.astype(np.float16)
    # y_test = y_test.astype(np.float16)
    print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
    for model in models:
            # start timer
            start = time.time()
            model_name = model.__class__.__name__
            try:
                fitted_model = model.fit(x_train, y_train)
                raw_score = fitted_model.score(x_test, y_test)
                lenient_score = lenient_accuracy(y_test, fitted_model.predict(x_test))
                score_k = cross_val_score(model,
                                        x_train,
                                        y_train,
                                        cv=k_fold,scoring=lenient_scorer)
            
                # end timer
                end = time.time()
                result = {
                    'model_name': model_name,
                    'duration': round(end - start, 2),
                    'k_fold_lenient': round(np.mean(score_k),4),
                    'lenient_score': round(lenient_score, 4),
                    'raw_score': round(raw_score, 4),
                    'comment': '',
                    'model': fitted_model
                    }
                s = pickle.dumps(fitted_model)
                if save:
                    Path(os.path.join(DATASET_DIR, f'{sym}')).mkdir(parents = True, exist_ok=True)
                    dump(result, os.path.join(DATASET_DIR, f'{sym}', f'predictor_{prediction_type}_{model_name}.joblib'))
                    print(f'saved {model_name} predictor_info.joblib')
            except Exception as e:
                print(e)
                # end timer
                end = time.time()
                result = {
                    'model_name': f'{model_name}_{prediction_type}',
                    'duration': round(end - start, 2),
                    'k_fold_lenient': -1,
                    'lenient_score': -1,
                    'raw_score': -1,
                    'comment': e,
                }
            finally:
                results.append(result)
                print(result)    

(1021888, 73) (255473, 73) (1021888,) (255473,)
saved LinearRegression predictor_info.joblib
{'model_name': 'LinearRegression', 'duration': 16.5, 'k_fold_lenient': 0.6838, 'lenient_score': 0.6819, 'raw_score': 0.9088, 'comment': '', 'model': LinearRegression()}


c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.8239e-25): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.77258e-25): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.78116e-25): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.71965e-25): result may not be accurate.
  return linalg.so

saved Ridge predictor_info.joblib
{'model_name': 'Ridge', 'duration': 3.12, 'k_fold_lenient': 0.6839, 'lenient_score': 0.6819, 'raw_score': 0.9088, 'comment': '', 'model': Ridge()}


c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.293e+08, tolerance: 5.326e+05
  model = cd_fast.enet_coordinate_descent(
c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.814e+08, tolerance: 4.246e+05
  model = cd_fast.enet_coordinate_descent(
c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the 

saved Lasso predictor_info.joblib
{'model_name': 'Lasso', 'duration': 393.62, 'k_fold_lenient': 0.6861, 'lenient_score': 0.6845, 'raw_score': 0.9091, 'comment': '', 'model': Lasso()}


...

saved Ridge predictor_info.joblib
{'model_name': 'Ridge', 'duration': 3.34, 'k_fold_lenient': 0.4674, 'lenient_score': 0.4657, 'raw_score': 0.9444, 'comment': '', 'model': Ridge()}


c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.264e+08, tolerance: 4.185e+05
  model = cd_fast.enet_coordinate_descent(
c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.861e+07, tolerance: 3.342e+05
  model = cd_fast.enet_coordinate_descent(
c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the 

saved Lasso predictor_info.joblib
{'model_name': 'Lasso', 'duration': 210.02, 'k_fold_lenient': 0.4662, 'lenient_score': 0.4641, 'raw_score': 0.9443, 'comment': '', 'model': Lasso()}


c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.261e+08, tolerance: 4.185e+05
  model = cd_fast.enet_coordinate_descent(
c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.840e+07, tolerance: 3.342e+05
  model = cd_fast.enet_coordinate_descent(
c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the 

saved ElasticNet predictor_info.joblib
{'model_name': 'ElasticNet', 'duration': 140.94, 'k_fold_lenient': 0.4659, 'lenient_score': 0.4636, 'raw_score': 0.9443, 'comment': '', 'model': ElasticNet()}
saved DecisionTreeRegressor predictor_info.joblib
{'model_name': 'DecisionTreeRegressor', 'duration': 481.13, 'k_fold_lenient': 0.9849, 'lenient_score': 0.9871, 'raw_score': 1.0, 'comment': '', 'model': DecisionTreeRegressor()}
saved ExtraTreeRegressor predictor_info.joblib
{'model_name': 'ExtraTreeRegressor', 'duration': 56.04, 'k_fold_lenient': 0.9789, 'lenient_score': 0.9819, 'raw_score': 0.9997, 'comment': '', 'model': ExtraTreeRegressor()}
saved HistGradientBoostingRegressor predictor_info.joblib
{'model_name': 'HistGradientBoostingRegressor', 'duration': 43.49, 'k_fold_lenient': 0.4557, 'lenient_score': 0.4526, 'raw_score': 0.9981, 'comment': '', 'model': HistGradientBoostingRegressor()}


In [8]:
# sort results by k_fold_lenient and lenient_score
result = pd.DataFrame(results)
result = result.sort_values(by=['k_fold_lenient', 'duration', 'lenient_score'], ascending=False)
# print(result)
results_without_error = result[result['k_fold_lenient'] != -1]
print(results_without_error)

                       model_name  duration  k_fold_lenient  lenient_score  \
11          DecisionTreeRegressor    481.13          0.9849         0.9871   
4           DecisionTreeRegressor    498.88          0.9846         0.9866   
5              ExtraTreeRegressor     55.90          0.9801         0.9839   
12             ExtraTreeRegressor     56.04          0.9789         0.9819   
2                           Lasso    393.62          0.6861         0.6845   
3                      ElasticNet    188.35          0.6861         0.6847   
1                           Ridge      3.12          0.6839         0.6819   
0                LinearRegression     16.50          0.6838         0.6819   
6   HistGradientBoostingRegressor     41.91          0.5543         0.5577   
7                LinearRegression     18.24          0.4674         0.4657   
8                           Ridge      3.34          0.4674         0.4657   
9                           Lasso    210.02          0.4662     